<a href="https://colab.research.google.com/github/JT-3710A/Neural-Network--Workshop/blob/main/notebook/neural_network_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision

In [ ]:
from torchvision import transforms as transform
transform=transform.Compose([transform.ToTensor()])

In [ ]:
train_set=torchvision.datasets.FashionMNIST(root="data",train=True,download=True,transform=transform)

In [ ]:
test_set=torchvision.datasets.FashionMNIST(root="data",train=False,download=True,transform=transform)

In [ ]:
train_set

In [ ]:
#data loader
from torch.utils.data import DataLoader

In [ ]:
#now making train and test data loader
train_loader=DataLoader(train_set,batch_size=32,shuffle=True)
test_loader=DataLoader(test_set,batch_size=32,shuffle=True)

In [ ]:
data=iter(test_loader)
img,labels=next(iter(test_loader))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

plt.figure(figsize=(10, 4))
imshow(torchvision.utils.make_grid(images, nrow=16))


In [ ]:
from torch import nn

In [ ]:
class jonishModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc_1=nn.Linear(28*28,512)
    self.fc_2=nn.Linear(512,256)
    self.fc_3=nn.Linear(256,10)
    self.relu=nn.ReLU()
    self.Flatten=nn.Flatten()

  def forward(self,x):
    x=self.Flatten(x)
    x=self.fc_1(x)
    x=self.relu(x)
    x=self.fc_2(x)
    x=self.relu(x)
    x=self.fc_3(x)
    return x

In [ ]:
model=jonishModel()

In [ ]:
!pip install torchinfo -q

In [ ]:
from torchinfo import summary
batch_size=64
summary(model,input_size=(batch_size,1,28,28))

In [ ]:
from torch import optim

#now calculating the loss called as criteriaon
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Move model to device once
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10

for epoch in range(epochs):
    model.train() # Set model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # Zero the gradients for each batch

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward() # Backpropagate the loss
        optimizer.step() # Update model weights

        running_loss += loss.item() # Accumulate loss

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader)

    # Evaluation phase
    model.eval() # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad(): # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            images = images.to(device) # Move test data to GPU
            labels = labels.to(device) # Move test labels to GPU
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {epoch_loss:.4f} - Test Accuracy: {accuracy:.2f}%")

print("Training complete!")